In [ ]:
!git clone https://github.com/ultralytics/yolov3

In [11]:
####need to insert '#' in front of Deloy layer and Pillow layer in 'requirments.txt'.
!cd yolov3;pip install -qr requirements.txt

In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
#Oxford pet dataset
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

In [ ]:
# Extract
!mkdir /content/data
!tar -xvf images.tar.gz -C /content/data
!tar -xvf annotations.tar.gz -C /content/data

In [ ]:
#make files for train, val dataset
!mkdir /content/ox_pet;
!cd /content/ox_pet; mkdir images; mkdir labels;
!cd /content/ox_pet/images; mkdir train; mkdir val
!cd /content/ox_pet/labels; mkdir train; mkdir val

In [ ]:
#convert data within trainval.txt to dataframe
import pandas as pd

pd.read_csv('/content/data/annotations/trainval.txt', sep=' ', header=None, names=['img_name', 'class_id', 'etc1', 'etc2'])

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
#create dataframe using for train and validation
def make_train_valid_df(list_filepath, img_dir, anno_dir, test_size=0.1):
  pet_df = pd.read_csv(list_filepath, sep=' ', header=None, names=['img_name', 'class_id', 'etc1', 'etc2'])
  pet_df['class_name'] = pet_df['img_name'].apply(lambda x:x[:x.rfind('_')])
  pet_df['img_filepath'] = img_dir + pet_df['img_name']+'.jpg'
  pet_df['anno_filepath'] = anno_dir + pet_df['img_name']+'.xml'
  pet_df = remove_no_annos(pet_df)


  train_df, val_df = train_test_split(pet_df, test_size=test_size, stratify=pet_df['class_id'], random_state=2021)
  return pet_df, train_df, val_df

def remove_no_annos(df):
  remove_rows = []
  for index, row in df.iterrows():
    anno_filepath = row['anno_filepath']
    if not os.path.exists(anno_filepath):
      print('##### index:', index, anno_filepath, 'is not exist. Delete from dataframe')
      remove_rows.append(index)
  df = df.drop(remove_rows, axis=0, inplace=False)
  return df


pet_df, train_df, val_df = make_train_valid_df('/content/data/annotations/trainval.txt',
                                               '/content/data/images/', '/content/data/annotations/xmls/', test_size=0.1)

In [ ]:
#check
pet_df.head()

In [ ]:
#set class name
CLASS_NAMES = pet_df['class_name'].unique().tolist()
print(CLASS_NAMES)

In [9]:
#convert annotation of Oxford pet data to format of Ultralytics yolo
import glob
import xml.etree.ElementTree as ET


def xml_to_txt(input_xml_file, output_txt_file, object_name):
  tree = ET.parse(input_xml_file)
  root = tree.getroot()
  img_node = root.find('size')

  if img_node is None:
    return None
  img_width = int(img_node.find('width').text)
  img_height = int(img_node.find('height').text)

  value_str = None
  with open(output_txt_file, 'w') as output_fpointer:
    for obj in root.findall('object'):
        xmlbox = obj.find('bndbox')
        x1 = int(xmlbox.find('xmin').text)
        y1 = int(xmlbox.find('ymin').text)
        x2 = int(xmlbox.find('xmax').text)
        y2 = int(xmlbox.find('ymax').text)
        if (x1 < 0) or (x2 < 0) or (y1 < 0) or (y2 < 0):
          break
        class_id, cx_norm, cy_norm, w_norm, h_norm = convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2)
        value_str = ('{0} {1} {2} {3} {4}').format(class_id, cx_norm, cy_norm, w_norm, h_norm)
        output_fpointer.write(value_str+'\n')


def convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2):
  class_id = CLASS_NAMES.index(object_name)
  center_x = (x1 + x2)/2
  center_y = (y1 + y2)/2
  width = x2 - x1
  height = y2 - y1
  center_x_norm = center_x / img_width
  center_y_norm = center_y / img_height
  width_norm = width / img_width
  height_norm = height / img_height

  return class_id, round(center_x_norm, 7), round(center_y_norm, 7), round(width_norm, 7), round(height_norm, 7)#############round는 소수점 7자리까지만 계산하라는 것.


In [ ]:
 class_id = CLASS_NAMES.index('yorkshire_terrier')
 print(class_id)

In [11]:
xml_to_txt('/content/data/annotations/xmls/Abyssinian_1.xml', '/content/ox_pet/labels/train/Abyssinian_1.txt', 'Abyssinian')

In [ ]:
train_df.head()

In [13]:
#create yolo annotation file
import shutil

def make_yolo_anno_file(df, tgt_images_dir, tgt_labels_dir):
  for index, row in df.iterrows():
    src_image_path = row['img_filepath']
    src_label_path = row['anno_filepath']
    object_name = row['class_name']
    target_label_path = tgt_labels_dir + row['img_name']+'.txt'
    shutil.copy(src_image_path, tgt_images_dir)
    xml_to_txt(src_label_path, target_label_path, object_name)


make_yolo_anno_file(train_df, '/content/ox_pet/images/train/', '/content/ox_pet/labels/train/')
make_yolo_anno_file(val_df, '/content/ox_pet/images/val/', '/content/ox_pet/labels/val/')

In [ ]:
#get yaml data made by Editplus
!wget -O /content/ox_pet/ox_pet.yaml https://raw.githubusercontent.com/pochune/research-codes/main/Codes_for_Image_detection/ox_pet.yaml

In [ ]:
import os, sys
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
#training
!cd /content/yolov3; python train.py --img 640 --batch 16 --epochs 20 --data /content/ox_pet/ox_pet.yaml --weights yolov3.pt --project=/mydrive/ultra_workdir --name pet --exist-ok

In [ ]:
#test
!cd yolov3;python detect.py --source /content/data/images/american_bulldog_101.jpg --weights /mydrive/ultra_workdir/pet/weights/best.pt --conf 0.2 --project=/content/data/output --name=run_image --exist-ok --line-thickness 2

In [ ]:
#vedio file download
!wget -O /content/data/ox_dog.mp4  https://github.com/pochune/research-codes/blob/main/Codes_for_Image_detection/ox_dog.mp4?raw=true

In [ ]:
#test video file
!cd yolov3; python detect.py --source /content/data/ox_dog.mp4 --weights /mydrive/ultra_workdir/pet/weights/best.pt --project /content/data/output --name run_video --exist-ok --line-thickness 3 --conf 0.4 --iou=0.6

In [ ]:
#create confusion matrix
!cd yolov3; python val.py --weights /mydrive/ultra_workdir/pet/weights/best.pt  --data /content/ox_pet/ox_pet.yaml --project /content/data/output --name=test_result --exist-ok --img 640 --iou 0.65

In [ ]:
Image(filename='/content/data/output/test_result/confusion_matrix.png', width=800)